In [1]:
import re
import requests
import time
from bs4 import BeautifulSoup as soup
import numpy as np
import pandas as pd
from statistics import mean 
from credentials import GOODREADS_KEY

In [2]:
authors = pd.read_csv("authors.csv")
authors.head()

,author_id,first_name,middle_name,last_name,birth_place,birth_date,gender,has_wikipedia,website,literaturinstitut,has_phd
0,0,Reto,NaN,Hänny,NaN,1947-04-13,male,True,NaN,NaN,NaN
1,1,Jost,NaN,Perfahl,NaN,NaN,male,False,NaN,NaN,NaN
2,2,Elisabeth,NaN,Plessen,NaN,1944-03-15,female,True,NaN,NaN,NaN
3,3,Vintila,NaN,Ivanceanu,NaN,1940-12-26,male,True,NaN,NaN,NaN
4,4,Horst,NaN,Bienek,NaN,1930-05-07,male,True,NaN,NaN,NaN


In [3]:
# most affective name concatenation
authors["full_name"] = (
    authors["first_name"]
    .str.cat(authors[["middle_name", "last_name"]].fillna("").astype(str), sep=" ")
    .str.replace("  ", " ")
)
authors.head()

,author_id,first_name,middle_name,last_name,birth_place,birth_date,gender,has_wikipedia,website,literaturinstitut,has_phd,full_name
0,0,Reto,NaN,Hänny,NaN,1947-04-13,male,True,NaN,NaN,NaN,Reto Hänny
1,1,Jost,NaN,Perfahl,NaN,NaN,male,False,NaN,NaN,NaN,Jost Perfahl
2,2,Elisabeth,NaN,Plessen,NaN,1944-03-15,female,True,NaN,NaN,NaN,Elisabeth Plessen
3,3,Vintila,NaN,Ivanceanu,NaN,1940-12-26,male,True,NaN,NaN,NaN,Vintila Ivanceanu
4,4,Horst,NaN,Bienek,NaN,1930-05-07,male,True,NaN,NaN,NaN,Horst Bienek


In [4]:
AUTHOR = "Reto Hänny"

# get author_id
author_url = f"https://www.goodreads.com/api/author_url/{AUTHOR}?key={GOODREADS_KEY}"
response = requests.get(author_url)
page = soup(response.text, "lxml-xml")
author_id = page.find("author")['id']

# get all author infos
# author_info_url = f"https://www.goodreads.com/author/show/{author_id}?format=xml&key={GOODREADS_KEY}"
# response = requests.get(author_info_url)
# page = soup(response.text, "lxml-xml")

# author_info_url = f"https://www.goodreads.com/author/list/{author_id}?format=xml&key={GOODREADS_KEY}"
# response = requests.get(author_info_url)
# page = soup(response.text, "lxml-xml")

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
author_page = page.find("link").text
response = requests.get(author_page, headers=headers)
page = soup(response.text, "html.parser")


In [5]:
page.find(attrs={"class": "authorName"}).text.strip()

'Reto Hänny'

In [6]:

tags = r"[a-zäöü’ ()]+"
snippet = 'Reto Hänny’s Followers'
snippet = snippet.lower()
followers = re.sub(tags, "", snippet)
print(followers)

# words = re.findall("([^\d]+|^)\d{0,5}[^\d]", 'Rainald Goetz’s Followers (40)', re.IGNORECASE)

In [7]:
genres = []
for div in page.find_all(attrs={"class": "dataItem"}):
    try:
        for link in div.find_all("a"):
            if "genres" in link['href']:
                genres.append(link.text)
    except:
        pass
authors_genres = "|".join(genres)
authors_genres

''

In [8]:
for div in page.find_all(attrs={"class": "dataTitle"}):
    if div.text == "Genre":
        for item in div.find_all(attrs={"class": "dataItem"}):
            print(item)

In [9]:
page.find(attrs={"class": "brownBackground"}).text

'Reto Hänny’s Followers'

In [10]:
page.find(attrs={"class": "count"}).next.next.next.next['href']

'/author/list/1425324.Reto_H_nny'

In [18]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
author_ids = []
author_names = []
goodreads_names = []
author_pubs = []
author_foll = []
author_avg = []
author_ratings = []
author_genres = []

for i, author in enumerate(authors.full_name.values):
    time.sleep(np.random.uniform(1,2))
    print("start iteration")
    try:
        # get author_id
        author_url = f"https://www.goodreads.com/api/author_url/{author}?key={GOODREADS_KEY}"
        response = requests.get(author_url)
        page = soup(response.text, "lxml-xml")
        author_page = page.find("link").text
        print("loaded_api")

        # load author_page
        response = requests.get(author_page, headers=headers)
        page = soup(response.text, "html.parser")
        print("loaded_author_page")

        # extract information
        avg_rating = page.find(attrs={"class": "average"}).text.strip()
        name = page.find(attrs={"class": "authorName"}).text.strip()
        total_ratings = page.find(attrs={"class": "votes"}).text.strip().replace(",", "")
        total_reviews = page.find(attrs={"class": "count"}).text.strip().replace(",", "")
        total_publications = page.find(attrs={"class": "count"}).next.next.next.next.next.strip().strip(" distinct works")
        follower_snippet = page.find(attrs={"class": "brownBackground"}).text.lower()
        tags = r"[a-zäöüßăšć.’\- ()]+"
        total_followers = re.sub(tags, "", follower_snippet).replace(",", "")
        genres = []
        for div in page.find_all(attrs={"class": "dataItem"}):
            try:
                for link in div.find_all("a"):
                    if "genres" in link['href']:
                        genres.append(link.text)
            except:
                pass
        authors_genres = "|".join(genres)
        authors_genres
        print(i, author, name, avg_rating, total_ratings, total_reviews, total_publications, total_followers, authors_genres)
        author_ids.append(i)
        author_names.append(author)
        goodreads_names.append(name)
        author_pubs.append(total_publications)
        author_foll.append(total_followers)
        author_avg.append(avg_rating)
        author_ratings.append(total_ratings)
        author_genres.append(authors_genres)
        print("appended_data")
    except:
        pass

df = pd.DataFrame({"author_id": author_ids, "author_name": author_names, "goodreads_name": goodreads_names, "goodreads_genre": author_genres, "total_publications": author_pubs, "goodreads_followers": author_foll, "average_rating": author_avg, "ratings_count": author_ratings})

start iteration
loaded_api
loaded_author_page
0 Reto Hänny Reto Hänny 3.33 9 3 9  
appended_data
start iteration
loaded_api
loaded_author_page
1 Jost Perfahl Jost Perfahl 3.2 5 3 9  
appended_data
start iteration
loaded_api
loaded_author_page
2 Elisabeth Plessen Elisabeth Plessen 3.23 43 8 8  
appended_data
start iteration
loaded_api
loaded_author_page
3 Vintila Ivanceanu Vintilă Ivănceanu 4.83 12 2 5  
appended_data
start iteration
loaded_api
loaded_author_page
4 Horst Bienek Horst Bienek 4.02 96 5 24 2 
appended_data
start iteration
loaded_api
loaded_author_page
5 Renate Schostack Renate Schostack 2.0 1 0 5  
appended_data
start iteration
loaded_api
loaded_author_page
6 Klaus Stiller Klaus Stiller 0.0 0 0 1  
appended_data
start iteration
loaded_api
loaded_author_page
7 Maria Elisabeth Straub Maria Elisabeth Straub 3.78 214 16 9  
appended_data
start iteration
loaded_api
loaded_author_page
8 Hans-Jürgen Fröhlich Hans Jürgen Fröhlich 0.0 0 0 1  
appended_data
start iteration
loaded_ap

loaded_api
loaded_author_page
73 Jurek Becker Jurek Becker 3.78 2899 182 34 21 Literature & Fiction
appended_data
start iteration
loaded_api
loaded_author_page
74 Peter Roos Peter De Roo 3.33 3 0 4  
appended_data
start iteration
loaded_api
loaded_author_page
75 Martin Roda Becher Martin Roda Becher 5.0 1 0 8 1 
appended_data
start iteration
loaded_api
loaded_author_page
76 Renate Rasp Renate Rasp 3.91 11 3 5  
appended_data
start iteration
loaded_api
loaded_author_page
77 Ingrid Puganigg Ingrid Puganigg 2.67 3 0 3  
appended_data
start iteration
loaded_api
loaded_author_page
78 Edwin Ortmann Edwin Ortmann 0.0 0 0 2 1 
appended_data
start iteration
loaded_api
loaded_author_page
79 Sten Nadolny Sten Nadolny 3.88 3037 139 17 32 Literature & Fiction
appended_data
start iteration
loaded_api
loaded_author_page
80 Klaus Merz Klaus Merz 3.23 124 12 25 3 
appended_data
start iteration
loaded_api
loaded_author_page
81 Jürgen Lodemann Jürgen Lodemann 3.77 13 1 16  
appended_data
start iteration


loaded_api
loaded_author_page
153 Bettina Blumenberg Bettina Blumenberg 3.69 26 3 12  
appended_data
start iteration
loaded_api
loaded_author_page
154 Gudrun Brug Dawn Baumann Brunke 4.39 112 19 18  Nonfiction
appended_data
start iteration
loaded_api
loaded_author_page
155 Jochen Missfeldt Jochen Missfeldt 3.86 21 4 9 2 
appended_data
start iteration
loaded_api
loaded_author_page
156 Bodo Morshäuser Bodo Morshäuser 3.6 10 0 9  
appended_data
start iteration
loaded_api
loaded_author_page
157 Alban Nikolai Herbst Alban Nikolai Herbst 4.0 19 4 11 5 Contemporary
appended_data
start iteration
loaded_api
loaded_author_page
158 Zsuzsanna Gahse Zsuzsanna Gahse 3.72 29 2 22  
appended_data
start iteration
loaded_api
loaded_author_page
159 Rainald Goetz Rainald Goetz 3.8 971 54 25 40 
appended_data
start iteration
loaded_api
loaded_author_page
160 Wolfgang Held Wolfgang Held 3.51 69 7 41  
appended_data
start iteration
loaded_api
loaded_author_page
161 Gerhard Köpf Gerhard Kopf 3.32 22 3 8  
app

start iteration
loaded_api
loaded_author_page
230 Daniel Grolle Daniel Grolle-Moscovici 0.0 0 0 6  
appended_data
start iteration
loaded_api
loaded_author_page
231 Evelyn Grill Evelyn Grill 3.71 38 5 13 1 Fiction
appended_data
start iteration
start iteration
loaded_api
loaded_author_page
233 Gerhard Falkner Gerhard Falkner 3.21 124 33 26  
appended_data
start iteration
loaded_api
loaded_author_page
234 Daniela Castner Daniela Castner 0.0 0 0 1  
appended_data
start iteration
start iteration
loaded_api
loaded_author_page
236 Thorsten Becker Thorsten Becker 3.77 26 1 17  
appended_data
start iteration
loaded_api
loaded_author_page
237 Joseph von Westphalen Joseph von Westphalen 3.57 93 8 32 1 
appended_data
start iteration
loaded_api
loaded_author_page
238 Martin Stephan Stephan Martin 4.3 40 8 10  
appended_data
start iteration
loaded_api
loaded_author_page
239 Manfred Seiler Heinz Lambertz 0.0 0 0 1  
appended_data
start iteration
loaded_api
loaded_author_page
240 Maria Seidemann Maria

loaded_author_page
307 Eleonore Frey Eleonore Frey 4.17 6 1 11  
appended_data
start iteration
loaded_api
loaded_author_page
308 Hubert Konrad Frank Jürgen Ferdinand Riemann 5.0 1 0 3  
appended_data
start iteration
start iteration
loaded_api
loaded_author_page
310 Georg Bydlinski Georg Bydlinski 3.97 73 11 19 1 
appended_data
start iteration
loaded_api
loaded_author_page
311 Christoph Buggert Christoph Buggert 0.0 0 0 4  
appended_data
start iteration
loaded_api
loaded_author_page
312 Pieke Biermann Pieke Biermann 3.89 733 52 34  
appended_data
start iteration
loaded_api
loaded_author_page
313 Peter Wawerzinek Peter Wawerzinek 3.37 101 15 15 4 
appended_data
start iteration
loaded_api
loaded_author_page
314 Ludger Schwarte Ludger Schwarte 3.33 6 0 19  
appended_data
start iteration
loaded_api
loaded_author_page
315 SAID  Edward W. Said 4.11 52069 3335 146 2365 Nonfiction
appended_data
start iteration
loaded_api
loaded_author_page
316 Theres Roth-Hunkeler Theres Roth-Hunkeler 0.0 0 0 1

...

start iteration
loaded_api
loaded_author_page
458 Thor Kunkel Thor Kunkel 3.62 86 7 15 1 Fiction
appended_data
start iteration
loaded_api
loaded_author_page
459 Ursula Fricker Ursula Fricker 2.83 12 1 6  
appended_data
start iteration
loaded_api
loaded_author_page
460 Ruth Erat Ruth Erat 0.0 0 0 1  
appended_data
start iteration
loaded_api
loaded_author_page
461 Terézia Mora Terézia Mora 3.72 690 90 23 é19 
appended_data
start iteration
loaded_api
loaded_author_page
462 Thomas Jonigk Thomas Jonigk 3.07 14 0 5  
appended_data
start iteration
loaded_api
loaded_author_page
463 Monika Helfer Monika Helfer 3.73 245 44 20 3 
appended_data
start iteration
loaded_api
loaded_author_page
464 Vladimir Vertlib Vladimir Vertlib 3.64 140 6 12 2 
appended_data
start iteration
loaded_api
loaded_author_page
465 Gudrun Seidenauer Gudrun Seidenauer 3.85 13 0 4 12 
appended_data
start iteration
loaded_api
loaded_author_page
466 Christian Uetz Christian Uetz 3.0 2 0 7  
appended_data
start iteration
loaded

start iteration
loaded_api
loaded_author_page
531 Guy Helminger Guy Helminger 3.15 60 5 11 8 
appended_data
start iteration
loaded_api
loaded_author_page
532 Roswitha Haring Roswitha Haring 2.71 7 4 3 6 
appended_data
start iteration
loaded_api
loaded_author_page
533 Arne Roß Ross H. Arnett Jr. 4.41 49 6 15 1 
appended_data
start iteration
loaded_api
loaded_author_page
534 Melinda Nadj Abonji Melinda Nadj Abonji 3.65 626 67 4 13 
appended_data
start iteration
loaded_api
loaded_author_page
535 Uwe Tellkamp Uwe Tellkamp 3.54 850 88 13 14 
appended_data
start iteration
loaded_api
loaded_author_page
536 Andreas Münzner Andreas Münzner 3.9 10 3 5  
appended_data
start iteration
loaded_api
loaded_author_page
537 Richard David Precht Richard David Precht 3.85 5784 398 25 242 Fiction|Nonfiction|Philosophy
appended_data
start iteration
loaded_api
loaded_author_page
538 Thomas Raab Thomas Raab 3.65 985 113 22 14 Mystery & Thrillers
appended_data
start iteration
loaded_api
loaded_author_page
539 

loaded_author_page
602 Dagrun Hintze Dagrun Hintze 3.0 3 0 2  
appended_data
start iteration
loaded_api
loaded_author_page
603 Sudabeh Mohafez Sudabeh Mohafez 3.83 64 6 10 1 Contemporary
appended_data
start iteration
loaded_api
loaded_author_page
604 Heike Geißler Heike Geissler 3.78 369 45 4 10 
appended_data
start iteration
loaded_api
loaded_author_page
605 Markus Orths Markus Orths 3.42 821 90 26 6 Literature & Fiction|Historical Fiction
appended_data
start iteration
loaded_api
loaded_author_page
606 Patrick Findeis Patrick Findeis 2.75 4 0 5 1 
appended_data
start iteration
loaded_api
loaded_author_page
607 Martin von Arndt Michelle Tea 3.8 21359 2179 38 736 Literature & Fiction|Biographies & Memoirs|Gay & Lesbian
appended_data
start iteration
loaded_api
loaded_author_page
608 Angelika Reitzer Angelika Reitzer 3.33 15 2 7 1 
appended_data
start iteration
loaded_api
loaded_author_page
609 Clemens J. Setz Clemens J. Setz 3.6 926 104 24 52 Fiction
appended_data
start iteration
loaded_

start iteration
loaded_api
loaded_author_page
675 Verena Güntner Verena Güntner 3.13 105 18 2  
appended_data
start iteration
loaded_api
loaded_author_page
676 Nadine Kegele Nadine Kegele 4.19 26 4 6 3 
appended_data
start iteration
loaded_api
loaded_author_page
677 Joachim Meyerhoff Joachim Meyerhoff 4.06 5745 361 5 124 
appended_data
start iteration
loaded_api
loaded_author_page
678 Larissa Boehning Larissa Boehning 3.33 39 6 7 1 Literature & Fiction
appended_data
start iteration
loaded_api
loaded_author_page
679 Georg Petz Georg Petz 0.0 0 0 2  
appended_data
start iteration
loaded_api
loaded_author_page
680 Tex Rubinowitz Tex Rubinowitz 4.32 1885 139 24 6 
appended_data
start iteration
loaded_api
loaded_author_page
681 Katharina Gericke Katarina Gericke 4.0 1 1 1  
appended_data
start iteration
loaded_api
loaded_author_page
682 Romana Ganzoni Romana Ganzoni 3.0 7 1 3  
appended_data
start iteration
loaded_api
loaded_author_page
683 Michael Fehr Michael Fehr 3.93 86 12 15 1 
appende

loaded_author_page
751 Ronya Othmann Ronya Othmann 4.1 30 2 4  
appended_data
start iteration
loaded_api
loaded_author_page
752 Yannic Han Biao Federer Yannic Han Biao Federer 3.33 18 2 1  
appended_data
start iteration
loaded_api
loaded_author_page
753 Andrea Gerster Andrea Gerster 3.0 5 1 6  
appended_data
start iteration
loaded_api
loaded_author_page
754 Julia Jost Jost W. Kramer 4.0 1 0 49  
appended_data
start iteration
loaded_api
loaded_author_page
755 Silvia Tschui Silvia Tschui 4.5 6 0 1  
appended_data
start iteration
loaded_api
loaded_author_page
756 Sarah Wipauer Valerie Katrin Fritsch Valerie Katrin G. 0.0 0 0 1  
appended_data
start iteration
loaded_api
loaded_author_page
757 Katharina Schultens Katharina Schultens 3.0 3 0 3 3 
appended_data
start iteration
loaded_api
loaded_author_page
758 Levin Westermann Levin Westermann 4.14 7 0 3 2 
appended_data
start iteration
loaded_api
loaded_author_page
759 Carolina Schutti Carolina Schutti 3.05 56 5 8 1 
appended_data
start iter

In [20]:
df.set_index('author_id').to_csv('goodreads_results_all.csv')

In [36]:
df[df.author_name != df.goodreads_name]

,author_id,author_name,goodreads_name,goodreads_genre,total_publications,goodreads_followers,average_rating,ratings_count
3,3,Vintila Ivanceanu,Vintilă Ivănceanu,,5,,4.83,12
8,8,Hans-Jürgen Fröhlich,Hans Jürgen Fröhlich,,1,,0.0,0
10,10,Marie-Thérèse Kerschbaumer,Marie-Therese Kerschbaumer,,4,1,4.33,3
15,15,Dieter Kühn,Dieter Kuhn,,28,,4.12,119
23,23,O. P. Zier,O.P. Zier,,1,,2.14,7
...,...,...,...,...,...,...,...,...
709,746,Leander Fischer,Leander Fisher,,2,,0.0,0
711,749,Daniel Heitzler,Daniel Hertzler,,6,,3.0,4
716,754,Julia Jost,Jost W. Kramer,,49,,4.0,1
718,756,Sarah Wipauer,Valerie Katrin Fritsch Valerie Katrin G.,,1,,0.0,0


In [42]:
authors.groupby('birth_date')['author_id'].count().sort_values(ascending=False).head(50)

birth_date
1980-07-01    11
1983-07-01     7
1974-07-01     7
1970-07-01     7
1968-07-01     7
1959-07-01     6
1977-07-01     6
1976-07-01     5
1979-07-01     5
1971-07-01     5
1982-07-01     5
1988-07-01     5
1973-07-01     4
1984-07-01     4
1972-07-01     4
1963-07-01     4
1978-07-01     4
1960-07-01     4
1990-07-01     4
1957-07-01     3
1962-07-01     3
1958-07-01     3
1986-07-01     3
1975-07-01     3
1967-07-01     3
1985-07-01     3
1955-07-05     2
1930-02-25     2
1944-05-18     2
1949-10-15     2
1992-07-01     2
1944-07-01     2
1947-07-01     2
1955-08-25     2
1966-07-01     2
1939-07-01     2
1957-06-09     2
1964-11-30     2
1954-07-01     2
1964-07-01     2
1961-09-27     2
1947-10-18     2
1952-07-01     2
1974-03-16     2
1946-08-18     1
1949-08-03     1
1949-06-10     1
1949-03-07     1
1949-07-16     1
1949-02-12     1
Name: author_id, dtype: int64

In [ ]:
total_publications = page.find("works_count").text
birth_date = page.find("born_at").text
author_followers = page.find("author_followers_count").text
print(f"{AUTHOR.upper()}: ")
print(f"Total publications: {total_publications}")
print(f"Birth date: {birth_date}")
print(f"Followers on Goodreads: {author_followers}")
print("BOOKS: ")
average_ratings = []
ratings_counts = []
for book in page.find_all("book"):
    print(f"---{book.find('title').text}---")
    print("Average Rating: ", book.find("average_rating").text)
    average_ratings.append(float(book.find("average_rating").text))
    print("Ratings Count: ", book.find("ratings_count").text)
    ratings_counts.append(int(book.find("ratings_count").text))
average_rating = mean(average_ratings)
ratings_count = sum(ratings_counts)
df = pd.DataFrame({"author_id": [AUTHOR], "total_publications": [total_publications], "average_rating": [average_rating], "ratings_count": [ratings_count]})

In [59]:
df

,author_id,total_publications,average_rating,ratings_count
0,Yannic Han Biao Federer,1,3.33,18


In [82]:
author_ids = []
author_names = []
author_pubs = []
author_foll = []
author_avg = []
author_ratings = []
for i, author in enumerate(authors.full_name.values):
    time.sleep(1)
    try:
        # get author_id
        author_url = f"https://www.goodreads.com/api/author_url/{author}?key={GOODREADS_KEY}"
        response = requests.get(author_url)
        page = soup(response.text, "lxml-xml")
        author_id = page.find("author")['id']
        
        # load author infos page
        author_info_url = f"https://www.goodreads.com/author/show/{author_id}?format=xml&key={GOODREADS_KEY}"
        response = requests.get(author_info_url)
        page = soup(response.text, "lxml-xml")

        # get all author infos
        total_publications = page.find("works_count").text
        birth_date = page.find("born_at").text
        author_followers = page.find("author_followers_count").text
        print(f"{author.upper()}: ")
        print(f"Total publications: {total_publications}")
        print(f"Birth date: {birth_date}")
        print(f"Followers on Goodreads: {author_followers}")
        print("BOOKS: ")
        average_ratings = []
        ratings_counts = []
        for book in page.find_all("book"):
            print(f"---{book.find('title').text}---")
            print("Average Rating: ", book.find("average_rating").text)
            average_ratings.append(float(book.find("average_rating").text))
            print("Ratings Count: ", book.find("ratings_count").text)
            ratings_counts.append(int(book.find("ratings_count").text))
        average_rating = mean(average_ratings)
        ratings_count = sum(ratings_counts)
        author_ids.append(i)
        author_names.append(author)
        author_pubs.append(total_publications)
        author_foll.append(author_followers)
        author_avg.append(average_rating)
        author_ratings.append(ratings_count)
    except Exception:
        pass

df = pd.DataFrame({"author_id": author_ids, "author_name": author_names, "total_publications": author_pubs, "goodreads_followers": author_foll, "average_rating": author_avg, "ratings_count": author_ratings})

RETO HÄNNY: 
Total publications: 9
Birth date: 1947/04/13
Followers on Goodreads: 0
BOOKS: 
---Zürich, Anfang September---
Average Rating:  3.50
Ratings Count:  2
---Blooms Schatten---
Average Rating:  2.50
Ratings Count:  2
---Sturz: Das dritte Buch vom Flug---
Average Rating:  4.00
Ratings Count:  1
---Am Boden Des Kopfes: Verwirrungen Eine Mitteleuropaers In Mitteleuropa---
Average Rating:  1.00
Ratings Count:  1
---Blooms Schatten---
Average Rating:  0.0
Ratings Count:  0
---Ruch: Ein Bericht---
Average Rating:  0.0
Ratings Count:  0
---Helldunkel: Ein Bilderbuch---
Average Rating:  0.0
Ratings Count:  0
---Zür(e)ich Brennt---
Average Rating:  4.33
Ratings Count:  3
---Konkursbuch. Zeitschrift für Vernunftkritik / Reisen ans Ende der Sonne: Revolten & Revolutionen---
Average Rating:  0.0
Ratings Count:  0
JOST PERFAHL: 
Total publications: 9
Birth date: 
Followers on Goodreads: 0
BOOKS: 
---Tiere und Tiergeschichten aller Völker---
Average Rating:  3.00
Ratings Count:  1
---Was Man

DIETER KÜHN: 
Total publications: 28
Birth date: 
Followers on Goodreads: 0
BOOKS: 
---The Age of Confucian Rule: The Song Transformation of China---
Average Rating:  4.06
Ratings Count:  100
---Der Parzival Des Wolfram Von Eschenbach---
Average Rating:  4.67
Ratings Count:  6
---Science and Civilisation in China: Volume 5, Chemistry and Chemical Technology, Part 9, Textile Technology: Spinning and Reeling---
Average Rating:  4.67
Ratings Count:  3
---Frau Merian! Eine Lebensgeschichte---
Average Rating:  4.00
Ratings Count:  2
---Clara Schumann, Klavier: Ein Lebensbuch---
Average Rating:  3.50
Ratings Count:  2
---Ich Wolkenstein: E. Biographie---
Average Rating:  5.00
Ratings Count:  1
---Josephine: Aus der öffentlichen Biographie der Josephine Baker.---
Average Rating:  4.00
Ratings Count:  1
---Rugby. Kampf in Gasse und Gedränge.---
Average Rating:  3.00
Ratings Count:  1
---Neue Fischer Weltgeschichte. Band 13: Ostasien bis 1800---
Average Rating:  0.0
Ratings Count:  0
---Markt C

ERIKA PEDRETTI: 
Total publications: 9
Birth date: 
Followers on Goodreads: 4
BOOKS: 
---Valerie oder Das unerzogene Auge.---
Average Rating:  2.86
Ratings Count:  7
---Engste Heimat---
Average Rating:  3.00
Ratings Count:  4
---Cizí domov---
Average Rating:  2.50
Ratings Count:  2
---fremd genug---
Average Rating:  4.00
Ratings Count:  1
---Kuckuckskind, Oder, Was Ich Ihr Unbedingt Noch Sagen Wollte: Roman---
Average Rating:  4.00
Ratings Count:  1
---Engste Heimat.---
Average Rating:  0.0
Ratings Count:  0
---Stones; Or, the Destruction of the Child Karl and Other Characters---
Average Rating:  0.0
Ratings Count:  0
---SonnenaufgäNge, SonnenuntergäNge: ErzäHlungen---
Average Rating:  0.0
Ratings Count:  0
---Begegnungen. Eine Festschrift für Max Frisch zum siebzigsten Geburtstag---
Average Rating:  0.0
Ratings Count:  0
HANNS-JOSEF ORTHEIL: 
Total publications: 62
Birth date: 1951/11/05
Followers on Goodreads: 26
BOOKS: 
---Die Erfindung des Lebens---
Average Rating:  4.06
Ratings

JEANNIE EBNER: 
Total publications: 66
Birth date: 
Followers on Goodreads: 2620
BOOKS: 
---Jonathan Livingston Seagull---
Average Rating:  3.84
Ratings Count:  198043
---Illusions: The Adventures of a Reluctant Messiah---
Average Rating:  4.15
Ratings Count:  56056
---One---
Average Rating:  3.95
Ratings Count:  12191
---The Bridge Across Forever: A True Love Story---
Average Rating:  3.89
Ratings Count:  11094
---There's No Such Place As Far Away---
Average Rating:  3.94
Ratings Count:  3701
---Messiah's Handbook: Reminders for the Advanced Soul---
Average Rating:  4.25
Ratings Count:  2072
---Running from Safety: An Adventure of the Spirit---
Average Rating:  3.80
Ratings Count:  2086
---A Gift Of Wings---
Average Rating:  3.79
Ratings Count:  1975
---Biplane---
Average Rating:  3.73
Ratings Count:  1823
---Stranger to the Ground---
Average Rating:  3.76
Ratings Count:  1157
CATARINA CARSTEN: 
Total publications: 4
Birth date: 
Followers on Goodreads: 0
BOOKS: 
---Meine Hoffnung hat

MARTIN LUKSAN: 
Total publications: 6
Birth date: 
Followers on Goodreads: 0
BOOKS: 
---Oni: Histoires du Japon d'autrefois---
Average Rating:  0.0
Ratings Count:  0
---Fuyu: Balades Au Japon---
Average Rating:  0.0
Ratings Count:  0
---Tsuyu: Les cinq saisons du senseï---
Average Rating:  0.0
Ratings Count:  0
---Aki: Les cinq saisons du senseï---
Average Rating:  0.0
Ratings Count:  0
---Fuyu: Les cinq saisons du senseî---
Average Rating:  0.0
Ratings Count:  0
---Haru: Les cinq saisons du senseï---
Average Rating:  0.0
Ratings Count:  0
KLAUS KONJETZKY: 
Total publications: 3
Birth date: 
Followers on Goodreads: 1
BOOKS: 
---La celălalt capăt al zilei---
Average Rating:  3.20
Ratings Count:  5
---Die stillenden Väter---
Average Rating:  0.0
Ratings Count:  0
---Was interessiert mich Goethes Geliebte?---
Average Rating:  0.0
Ratings Count:  0
PAUL KERSTEN: 
Total publications: 6
Birth date: 
Followers on Goodreads: 18
BOOKS: 
---Scrum: A Pocket Guide: A Smart Travel Companion---
Aver

...

PETER TRUSCHNER: 
Total publications: 6
Birth date: 
Followers on Goodreads: 0
BOOKS: 
---Die Träumer---
Average Rating:  3.50
Ratings Count:  4
---Das fünfunddreißigste Jahr---
Average Rating:  2.17
Ratings Count:  6
---Schlangenkind Roman---
Average Rating:  2.50
Ratings Count:  2
---Peter Truschner - Bangkok Struggle---
Average Rating:  2.00
Ratings Count:  1
---Das fünfunddreißigste Jahr: Roman---
Average Rating:  0.0
Ratings Count:  0
---Johannes Steidl: Nero D'Inferno---
Average Rating:  0.0
Ratings Count:  0
VALERIE FRITSCH: 
Total publications: 7
Birth date: 
Followers on Goodreads: 4
BOOKS: 
---Winters Garten---
Average Rating:  3.54
Ratings Count:  159
---Herzklappen von Johnson & Johnson---
Average Rating:  3.56
Ratings Count:  45
---Die Welt ist meine Innerei: Reisebriefe und Bilder---
Average Rating:  3.50
Ratings Count:  4
---Die Verkörperungen---
Average Rating:  3.67
Ratings Count:  3
---kinder der unschärferelation---
Average Rating:  2.50
Ratings Count:  2
---Winters 

MARKO DINIĆ: 
Total publications: 1
Birth date: 
Followers on Goodreads: 0
BOOKS: 
---Die guten Tage---
Average Rating:  3.69
Ratings Count:  49
SASCHA MACHT: 
Total publications: 3
Birth date: 
Followers on Goodreads: 1
BOOKS: 
---Der Krieg im Garten des Königs der Toten---
Average Rating:  3.06
Ratings Count:  17
---Der Krieg im Garten des Königs der Toten: Roman---
Average Rating:  3.00
Ratings Count:  1
---Schreiben lernen im Sozialismus: Das Institut für Literatur "Johannes R. Becher"---
Average Rating:  0.0
Ratings Count:  0
STEFANIE SARGNAGEL: 
Total publications: 6
Birth date: 
Followers on Goodreads: 24
BOOKS: 
---Statusmeldungen---
Average Rating:  4.22
Ratings Count:  102
---Binge Living---
Average Rating:  4.05
Ratings Count:  101
---Fitness---
Average Rating:  4.11
Ratings Count:  64
---In der Zukunft sind wir alle tot: Neue Callcenter-Monologe---
Average Rating:  3.81
Ratings Count:  63
---Der allerletzte Tag der Menschheit---
Average Rating:  5.00
Ratings Count:  1
---Re

ANSELM NEFT: 
Total publications: 12
Birth date: 
Followers on Goodreads: 3
BOOKS: 
---Die bessere Geschichte---
Average Rating:  4.46
Ratings Count:  13
---Helden in Schnabelschuhen---
Average Rating:  4.14
Ratings Count:  7
---Hell---
Average Rating:  3.43
Ratings Count:  7
---Die Lebern der Anderen---
Average Rating:  3.00
Ratings Count:  2
---Götter, Gurus und Gestörte---
Average Rating:  3.00
Ratings Count:  1
---Helden in Schnabelschuhen: Roman---
Average Rating:  0.0
Ratings Count:  0
---Vorsicht Volk!: Oder: Bewegungen im Wahn?---
Average Rating:  4.33
Ratings Count:  6
---Komma zum Punkt---
Average Rating:  3.50
Ratings Count:  2
---EXOT #16. Zeitschrift für komische Literatur---
Average Rating:  5.00
Ratings Count:  1
---Komma zum Punkt: Slamtexte aus der Hauptstadt---
Average Rating:  4.00
Ratings Count:  1
BOV BJERG: 
Total publications: 10
Birth date: 
Followers on Goodreads: 26
BOOKS: 
---Auerhaus---
Average Rating:  3.69
Ratings Count:  1686
---Serpentinen---
Average Rat

BIRGIT BIRNBACHER: 
Total publications: 4
Birth date: 
Followers on Goodreads: 1
BOOKS: 
---Ich an meiner Seite---
Average Rating:  4.04
Ratings Count:  24
---Wir ohne Wal---
Average Rating:  3.14
Ratings Count:  7
---Mal lichterloh, mal wasserblau: Ein Kinderbuch zum Thema Autismus---
Average Rating:  3.00
Ratings Count:  1
---BeziehungKrisenHerd: 19. Münchner Kurzgeschichtenwettbewerb---
Average Rating:  0.0
Ratings Count:  0
RONYA OTHMANN: 
Total publications: 4
Birth date: 
Followers on Goodreads: 0
BOOKS: 
---Die Sommer---
Average Rating:  5.00
Ratings Count:  1
---Flexen. Flâneusen* schreiben Städte---
Average Rating:  3.81
Ratings Count:  21
---HAYMATLOS - Gedichte---
Average Rating:  4.75
Ratings Count:  8
---FLEXEN: Flâneusen* schreiben Städte---
Average Rating:  0.0
Ratings Count:  0
YANNIC HAN BIAO FEDERER: 
Total publications: 1
Birth date: 
Followers on Goodreads: 0
BOOKS: 
---Und alles wie aus Pappmaché---
Average Rating:  3.33
Ratings Count:  18
ANDREA GERSTER: 
Total pu

LAURA FREUDENTHALER: 
Total publications: 3
Birth date: 
Followers on Goodreads: 3
BOOKS: 
---Die Königin schweigt---
Average Rating:  3.88
Ratings Count:  56
---Geistergeschichte---
Average Rating:  3.12
Ratings Count:  24
---Der Schädel von Madeleine. Paargeschichten---
Average Rating:  2.00
Ratings Count:  2


In [84]:
df.set_index('author_id').to_csv('goodreads_results.csv')

In [90]:
df.sort_values('ratings_count', ascending=False).head(20)

,author_id,author_name,total_publications,goodreads_followers,average_rating,ratings_count
320,343,Wolfgang Herrmann,1691,730860,4.133000,4844055
492,524,Henning Ahrens,56,65657,4.199000,1263451
516,548,Thomas Lang,1979,6708,4.152000,870681
43,44,Jeannie Ebner,66,2620,3.910000,290198
66,67,Maria Felsenreich,339,4334,3.898000,244512
290,312,Pieke Biermann,34,0,2.265000,165475
406,436,Leopold Federmair,55,4743,3.677000,134450
199,212,Peter Grosz,21,1211,3.834000,126093
661,697,FALKNER,38,3129,4.219000,97180
69,70,Helmut Degner,13,0,3.939000,94452
